# Imports and Setup

In [ ]:
import os
import re
import gc
import glob
import imageio
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
print('TF version: ', tf.__version__)
from tensorflow.keras.layers import *
from tensorflow.keras.models import *

from sklearn.model_selection import train_test_split

Set up Weights and Biases

In [ ]:
import wandb
print('W&B version: ', wandb.__version__)
from wandb.keras import WandbCallback
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_key")
wandb.login(key = secret_value_0)

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

# Prepare Dataset

There are four sub-directories per patient corresponding to different MRI Image Sequencing methods. We are using FLAIR.

In [ ]:
# Load training csv file
df = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv')

def get_patient_id(patient_id):
    if patient_id < 10:
        return '0000'+str(patient_id)
    elif patient_id >= 10 and patient_id < 100:
        return '000'+str(patient_id)
    elif patient_id >= 100 and patient_id < 1000:
        return '00'+str(patient_id)
    else:
        return '0'+str(patient_id)

def get_path(row):
    patient_id = get_patient_id(row.BraTS21ID)
    return f'../input/rsna-miccai-png/train/{patient_id}/FLAIR/'

df['path'] = df.apply(lambda row: get_path(row), axis=1)

# Removing two patient ids from the dataframe since there are not FLAIR directories for these ids. 
df = df.loc[df.BraTS21ID!=109]
df = df.loc[df.BraTS21ID!=709]
df = df.reset_index(drop=True)

df.head()

Prepare train-test split. Note that there are only 585 patients so if you are doing video classification, K-fold training might be beneficial. 

In [ ]:
train_df, valid_df = train_test_split(df, test_size=0.1, stratify=df.MGMT_value.values)
print(f'Size of train_df: {len(train_df)}; valid_df: {len(valid_df)}')

In [ ]:
CONFIG = dict(
    NUM_FRAMES = 10,
    BATCH_SIZE = 8,
    EPOCHS = 100,
    IMG_SIZE = 224,
    LSTM_UNITS = 256,
    competition = 'rsna-miccai-brain',
    _wandb_kernel = 'ayut'
)

In [ ]:

def sorted_nicely(l): 
    """ Sort the given iterable in the way that humans expect.""" 
    convert = lambda text: int(text) if text.isdigit() else text 
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(l, key = alphanum_key)

In [ ]:
def decode_image(image):
    # convert the compressed string to a 3D uint8 tensor
    image = tf.image.decode_png(image, channels=1)
    # Normalize image
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    
    return image

def parse_frames(dirname):
    # get MRI images file paths for given patient 
    paths = glob.glob(dirname.decode('utf8')+'/*.png')
    # Sort the images to get sequential imaging
    paths = sorted_nicely(paths)
    
    # randomly select a window of images to be used as sequence
    start = tf.random.uniform((1,), maxval=len(paths)-CONFIG['NUM_FRAMES'], dtype=tf.int32)

    paths = tf.slice(paths, start, [CONFIG['NUM_FRAMES']])
    
    def get_frames(path):
        # Load image
        image = tf.io.read_file(path)
        image = decode_image(image)
        # Resize image
        image = tf.image.resize(image, (CONFIG['IMG_SIZE'], CONFIG['IMG_SIZE']))
        
        return image

    mri_images = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn=get_frames, elems=paths, fn_output_signature=tf.float32))
    
    return mri_images
    
def load_frame(df_dict):
    dirname = df_dict['path']
    paths = tf.numpy_function(parse_frames, [dirname], tf.float32)
    
    # Parse label
    label = df_dict['MGMT_value']
    label = tf.cast(label, tf.float32)
    
    return paths, label

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

trainloader = tf.data.Dataset.from_tensor_slices(dict(train_df))
validloader = tf.data.Dataset.from_tensor_slices(dict(valid_df))


trainloader = (
    trainloader
    .shuffle(1024)
    .map(load_frame, num_parallel_calls=AUTOTUNE)
    .batch(CONFIG['BATCH_SIZE'])
    .prefetch(AUTOTUNE)
)

validloader = (
    validloader
    .map(load_frame, num_parallel_calls=AUTOTUNE)
    .batch(CONFIG['BATCH_SIZE'])
    .prefetch(AUTOTUNE)
)

In [ ]:
# test out the trainloader
frames, labels = next(iter(trainloader))

In order to visualize the samples from our trainloader, I am using W&B. I find it easier to log everything onto W&B to visualize data than to write Matplotlib code. 

In [ ]:
run = wandb.init(project='brain-tumor-video', job_type='dataloader-viz')

os.makedirs('gifs/')
for i, frame in enumerate(frames):
    imageio.mimsave(f'gifs/out_{i}.gif', (frame*255).numpy().astype('uint8'))    

wandb.log({'examples': [wandb.Image(f'gifs/out_{i}.gif', caption=f'{label.numpy()}') for i, label in enumerate(labels)]})
    
run.finish()

#  Model

In order to model both spatial and temporal nature of videos, we can use a hybrid of CNN + LSTM model. 

* The `FeatureExtractor` model uses an EfficientNetB0 model as CNN backbone. It will be used to model the spatial aspect of videos. <br>
* The `MRIModel` uses a `TimeDistributed` layer that runs the `FeatureExtractor` `NUM_FRAMES` times to get a vector of `(NUM_FRAMES, 1280)`. <br>
* This is then fed to a single LSTM layer. You can use GRU and even Transformer in place of LSTM. I have used 256 units as it gave me the best results. 

In [ ]:
def FeatureExtractor():
    base_model = tf.keras.applications.EfficientNetB0(include_top=False, weights='imagenet')
    base_model.trainabe = True

    inputs = Input((CONFIG['IMG_SIZE'], CONFIG['IMG_SIZE'], 1))
    x = Conv2D(3, kernel_size=(3, 3), padding='same', activation='relu')(inputs)
    x = base_model(x, training=True)
    flattened_output = GlobalAveragePooling2D()(x)
    
    return Model(inputs, flattened_output)

tf.keras.backend.clear_session()
model = FeatureExtractor()
model.summary()

In [ ]:
def MRIModel():
    inputs = Input((CONFIG['NUM_FRAMES'], CONFIG['IMG_SIZE'], CONFIG['IMG_SIZE'], 1))
    feature_extractor = FeatureExtractor()
    
    time_wrapper = TimeDistributed(feature_extractor)(inputs)
    
    lstm_out = LSTM(CONFIG['LSTM_UNITS'], return_sequences=True, name="lstm")(time_wrapper)
    outputs = Dense(1, activation='sigmoid', name="lstm_sigmoid")(lstm_out)
    
    return Model(inputs, outputs)

tf.keras.backend.clear_session() 
model = MRIModel()
model.summary()

# 🚅 Train

This is a simple training pipeline that uses early stopping as regularizer and `WandbCallback` to log the metrics to Weights and Biases.

In [ ]:
# Callbacks
earlystopper = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=5, verbose=0, mode='min',
    restore_best_weights=True
)

In [ ]:
tf.keras.backend.clear_session() 
model = MRIModel()
model.compile('adam', 'binary_crossentropy', metrics=['acc'])

run = wandb.init(project='brain-tumor-video', 
                 group='EffnetB0-LSTM-256', 
                 job_type='train', 
                 config=CONFIG)

# Train
_ = model.fit(trainloader, 
              epochs=CONFIG['EPOCHS'],
              validation_data=validloader,
              callbacks=[WandbCallback(),
                         earlystopper])

# Evaluate
loss, acc = model.evaluate(validloader)
wandb.log({'Val Accuracy': round(acc, 3)})

run.finish()

In [ ]:
CONFIG = dict(
    NUM_FRAMES = 10,
    BATCH_SIZE = 8,
    EPOCHS = 100,
    IMG_SIZE = 224,
    LSTM_UNITS = 128,
    competition = 'rsna-miccai-brain',
    _wandb_kernel = 'ayut'
)

In [ ]:
def MRIModel():
    inputs = Input((CONFIG['NUM_FRAMES'], CONFIG['IMG_SIZE'], CONFIG['IMG_SIZE'], 1))
    feature_extractor = FeatureExtractor()
    
    time_wrapper = TimeDistributed(feature_extractor)(inputs)
    
    lstm_out = LSTM(CONFIG['LSTM_UNITS'], return_sequences=True, name="lstm")(time_wrapper)
    outputs = Dense(1, activation='sigmoid', name="lstm_sigmoid")(lstm_out)
    
    return Model(inputs, outputs)

tf.keras.backend.clear_session() 
model = MRIModel()
model.summary()

In [ ]:
# Callbacks
earlystopper = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=5, verbose=0, mode='min',
    restore_best_weights=True
)

In [ ]:
tf.keras.backend.clear_session() 
model = MRIModel()
model.compile('adam', 'binary_crossentropy', metrics=['acc'])

run = wandb.init(project='brain-tumor-video', 
                 group='EffnetB0-LSTM-128', 
                 job_type='train', 
                 config=CONFIG)

# Train
_ = model.fit(trainloader, 
              epochs=CONFIG['EPOCHS'],
              validation_data=validloader,
              callbacks=[WandbCallback(),
                         earlystopper])

# Evaluate
loss, acc = model.evaluate(validloader)
wandb.log({'Val Accuracy': round(acc, 3)})

run.finish()

In [ ]:
CONFIG = dict(
    NUM_FRAMES = 10,
    BATCH_SIZE = 8,
    EPOCHS = 100,
    IMG_SIZE = 224,
    LSTM_UNITS = 512,
    competition = 'rsna-miccai-brain',
    _wandb_kernel = 'ayut'
)

In [ ]:
def MRIModel():
    inputs = Input((CONFIG['NUM_FRAMES'], CONFIG['IMG_SIZE'], CONFIG['IMG_SIZE'], 1))
    feature_extractor = FeatureExtractor()
    
    time_wrapper = TimeDistributed(feature_extractor)(inputs)
    
    lstm_out = LSTM(CONFIG['LSTM_UNITS'], return_sequences=True, name="lstm")(time_wrapper)
    outputs = Dense(1, activation='sigmoid', name="lstm_sigmoid")(lstm_out)
    
    return Model(inputs, outputs)

tf.keras.backend.clear_session() 
model = MRIModel()
model.summary()

In [ ]:
tf.keras.backend.clear_session() 
model = MRIModel()
model.compile('adam', 'binary_crossentropy', metrics=['acc'])

run = wandb.init(project='brain-tumor-video', 
                 group='EffnetB0-LSTM-512', 
                 job_type='train', 
                 config=CONFIG)

# Train
_ = model.fit(trainloader, 
              epochs=CONFIG['EPOCHS'],
              validation_data=validloader,
              callbacks=[WandbCallback(),
                         earlystopper])

# Evaluate
loss, acc = model.evaluate(validloader)
wandb.log({'Val Accuracy': round(acc, 3)})

run.finish()